In [1]:
%load_ext autoreload
%autoreload 2

from dotenv import load_dotenv
import pandas as pd
import os
import sys
import datetime

root_path = os.path.abspath(os.path.join('..'))
if root_path not in sys.path:
    sys.path.append(root_path)
    
load_dotenv()

True

In [2]:
import tensorflow as tf
tf.__version__, tf.config.list_physical_devices('GPU')

('2.10.1', [])

In [3]:
from training.utils import date_range, df_to_X_y

df_train = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2019, 11, 1), 
        datetime.date(2020, 2, 1),
        datetime.timedelta(days=1),
    )
])
df_val = pd.concat([
    pd.read_csv(f'../datasets/d{d.strftime("%Y-%m-%d")}.csv') for d in date_range(
        datetime.date(2020, 2, 1), 
        datetime.date(2020, 2, 15),
        datetime.timedelta(days=1),
    )
])

X_train, y_train = df_to_X_y(df_train)
X_val, y_val = df_to_X_y(df_val)

X_train.shape, y_train.shape, X_val.shape, y_val.shape

((1861099, 124), (1861099, 4), (316407, 124), (316407, 4))

In [4]:
from keras.models import Sequential, clone_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint
from keras.losses import MeanSquaredError
from keras.metrics import RootMeanSquaredError
from keras.optimizers import Adam


model = Sequential()
model.add(InputLayer((124, 1)))
model.add(Conv1D(64, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation=LeakyReLU(alpha=0.01)))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 122, 64)           256       
                                                                 
 flatten (Flatten)           (None, 7808)              0         
                                                                 
 dense (Dense)               (None, 8)                 62472     
                                                                 
 dense_1 (Dense)             (None, 1)                 9         
                                                                 
Total params: 62,737
Trainable params: 62,737
Non-trainable params: 0
_________________________________________________________________


In [5]:
model_view = clone_model(model)
cp_view = ModelCheckpoint('../model/d-view.keras', save_best_only=True)
model_view.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_view.fit(X_train, y_train[:, 0], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_view])

Epoch 1/10
58160/58160 [==============================] - 182s 3ms/step - loss: 18.0194 - root_mean_squared_error: 4.2449 - val_loss: 119.8957 - val_root_mean_squared_error: 10.9497
Epoch 2/10
58160/58160 [==============================] - 190s 3ms/step - loss: 16.5739 - root_mean_squared_error: 4.0711 - val_loss: 145.0649 - val_root_mean_squared_error: 12.0443
Epoch 3/10
58160/58160 [==============================] - 192s 3ms/step - loss: 15.9792 - root_mean_squared_error: 3.9974 - val_loss: 124.8833 - val_root_mean_squared_error: 11.1751
Epoch 4/10
58160/58160 [==============================] - 190s 3ms/step - loss: 15.5731 - root_mean_squared_error: 3.9463 - val_loss: 121.2960 - val_root_mean_squared_error: 11.0134
Epoch 5/10
58160/58160 [==============================] - 156s 3ms/step - loss: 15.3225 - root_mean_squared_error: 3.9144 - val_loss: 121.1577 - val_root_mean_squared_error: 11.0072
Epoch 6/10
58160/58160 [==============================] - 165s 3ms/step - loss: 15.4047 - 

In [6]:

model_cart = clone_model(model)
cp_cart = ModelCheckpoint('../model/d-cart.keras', save_best_only=True)
model_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_cart.fit(X_train, y_train[:, 1], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_cart])

Epoch 1/10
58160/58160 [==============================] - 209s 4ms/step - loss: 23.2139 - root_mean_squared_error: 4.8181 - val_loss: 47.5973 - val_root_mean_squared_error: 6.8991
Epoch 2/10
58160/58160 [==============================] - 213s 4ms/step - loss: 8.4487 - root_mean_squared_error: 2.9067 - val_loss: 47.7074 - val_root_mean_squared_error: 6.9071
Epoch 3/10
58160/58160 [==============================] - 224s 4ms/step - loss: 8.2198 - root_mean_squared_error: 2.8670 - val_loss: 48.6063 - val_root_mean_squared_error: 6.9718
Epoch 4/10
58160/58160 [==============================] - 237s 4ms/step - loss: 8.1374 - root_mean_squared_error: 2.8526 - val_loss: 52.5725 - val_root_mean_squared_error: 7.2507
Epoch 5/10
58160/58160 [==============================] - 172s 3ms/step - loss: 8.0539 - root_mean_squared_error: 2.8379 - val_loss: 51.8221 - val_root_mean_squared_error: 7.1988
Epoch 6/10
58160/58160 [==============================] - 174s 3ms/step - loss: 8.0637 - root_mean_squar

In [7]:

model_remove_from_cart = clone_model(model)
cp_remove_from_cart = ModelCheckpoint('../model/d-remove_from_cart.keras', save_best_only=True)
model_remove_from_cart.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_remove_from_cart.fit(X_train, y_train[:, 2], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_remove_from_cart])

Epoch 1/10
58160/58160 [==============================] - 167s 3ms/step - loss: 4.8194 - root_mean_squared_error: 2.1953 - val_loss: 46.6587 - val_root_mean_squared_error: 6.8307
Epoch 2/10
58160/58160 [==============================] - 165s 3ms/step - loss: 3.4737 - root_mean_squared_error: 1.8638 - val_loss: 46.8639 - val_root_mean_squared_error: 6.8457
Epoch 3/10
58160/58160 [==============================] - 166s 3ms/step - loss: 3.3083 - root_mean_squared_error: 1.8189 - val_loss: 47.2597 - val_root_mean_squared_error: 6.8746
Epoch 4/10
58160/58160 [==============================] - 160s 3ms/step - loss: 3.2338 - root_mean_squared_error: 1.7983 - val_loss: 46.6783 - val_root_mean_squared_error: 6.8322
Epoch 5/10
58160/58160 [==============================] - 160s 3ms/step - loss: 3.1973 - root_mean_squared_error: 1.7881 - val_loss: 47.9472 - val_root_mean_squared_error: 6.9244
Epoch 6/10
58160/58160 [==============================] - 163s 3ms/step - loss: 3.1782 - root_mean_square

In [8]:

model_purchase = clone_model(model)
cp_purchase = ModelCheckpoint('../model/d-purchase.keras', save_best_only=True)
model_purchase.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate=0.0001), metrics=[RootMeanSquaredError()])
model_purchase.fit(X_train, y_train[:, 3], validation_data=(X_val, y_val), epochs=10, callbacks=[cp_purchase])

Epoch 1/10
58160/58160 [==============================] - 166s 3ms/step - loss: 1.8542 - root_mean_squared_error: 1.3617 - val_loss: 52.3617 - val_root_mean_squared_error: 7.2361
Epoch 2/10
58160/58160 [==============================] - 166s 3ms/step - loss: 0.7300 - root_mean_squared_error: 0.8544 - val_loss: 51.6495 - val_root_mean_squared_error: 7.1868
Epoch 3/10
58160/58160 [==============================] - 166s 3ms/step - loss: 0.7206 - root_mean_squared_error: 0.8489 - val_loss: 52.3907 - val_root_mean_squared_error: 7.2381
Epoch 4/10
58160/58160 [==============================] - 167s 3ms/step - loss: 0.7075 - root_mean_squared_error: 0.8411 - val_loss: 51.2761 - val_root_mean_squared_error: 7.1607
Epoch 5/10
58160/58160 [==============================] - 175s 3ms/step - loss: 0.7084 - root_mean_squared_error: 0.8417 - val_loss: 51.6019 - val_root_mean_squared_error: 7.1834
Epoch 6/10
58160/58160 [==============================] - 180s 3ms/step - loss: 0.7001 - root_mean_square